In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
# A couple utilities
get_topics = lambda string: string.strip(">").strip("<").split("><")

In [3]:
#Take questions with tags
posts = pd.DataFrame.from_csv("postsCsTheory.csv")
posts = posts.reset_index(level=[0],drop=False)
posts = posts[posts.Tags.notnull()]
posts.head()

,PostId,CreationDate,PostType,OwnerUserId,Score,Views,Tags
0,2,2010-08-16T19:46:06.793,1,48,11,724,<ds.algorithms><total-ordering><sorting>
1,3,2010-08-16T19:50:13.663,1,35,5,593,<ds.algorithms><hamiltonian-paths>
2,4,2010-08-16T19:58:52.537,1,54,11,714,<np-hardness><set-cover><cc.complexity-theory>
3,5,2010-08-16T20:05:19.027,1,64,7,269,<ds.algorithms>
4,6,2010-08-16T20:05:51.693,1,47,5,163,<concurrency><process-algebra><formal-modeling>


In [4]:
posts_topics = posts.apply(lambda row:row.append(pd.Series([1]*len(get_topics(row.Tags)),index=get_topics(row.Tags))),axis=1)
posts_topics = posts_topics.fillna(0)
topics = posts_topics.iloc[:, 7:].sum(axis=0).sort_values(ascending=False)
top_perc = 0.05
top_topics = topics.iloc[:int(len(topics)*top_perc)]
top_topics.head()

cc.complexity-theory    2012
ds.algorithms           1375
reference-request       1039
graph-theory             996
graph-algorithms         659
dtype: float64

In [5]:
import sys
df_from_row = lambda row: pd.DataFrame(dict(zip(row.index,[[v] for v in row.values])))
def concat_set_tags(row,topics):
    replicated = pd.concat([df_from_row(row)]*len(topics),axis=0,ignore_index=True)
    replicated.loc[:,'TagSplit'] = pd.Series(topics)
    return replicated

replicate_for_topics = lambda row: concat_set_tags(row,get_topics(row.Tags))

rep_posts = pd.DataFrame()
for idx,row in posts.iterrows():
    rep_posts = rep_posts.append(replicate_for_topics(row),ignore_index=True)
rep_posts

,CreationDate,OwnerUserId,PostId,PostType,Score,Tags,Views,TagSplit
0,2010-08-16T19:46:06.793,48,2,1,11,<ds.algorithms><total-ordering><sorting>,724,ds.algorithms
1,2010-08-16T19:46:06.793,48,2,1,11,<ds.algorithms><total-ordering><sorting>,724,total-ordering
2,2010-08-16T19:46:06.793,48,2,1,11,<ds.algorithms><total-ordering><sorting>,724,sorting
3,2010-08-16T19:50:13.663,35,3,1,5,<ds.algorithms><hamiltonian-paths>,593,ds.algorithms
4,2010-08-16T19:50:13.663,35,3,1,5,<ds.algorithms><hamiltonian-paths>,593,hamiltonian-paths
5,2010-08-16T19:58:52.537,54,4,1,11,<np-hardness><set-cover><cc.complexity-theory>,714,np-hardness
6,2010-08-16T19:58:52.537,54,4,1,11,<np-hardness><set-cover><cc.complexity-theory>,714,set-cover
7,2010-08-16T19:58:52.537,54,4,1,11,<np-hardness><set-cover><cc.complexity-theory>,714,cc.complexity-theory
8,2010-08-16T20:05:19.027,64,5,1,7,<ds.algorithms>,269,ds.algorithms
9,2010-08-16T20:05:51.693,47,6,1,5,<concurrency><process-algebra><formal-modeling>,163,concurrency


## A couple checks

In [6]:
#1 Question, 2 Answer
posts[(posts.PostType!=1)&(posts.PostType!=2)]

,PostId,CreationDate,PostType,OwnerUserId,Score,Views,Tags


In [7]:
posts[(posts.OwnerUserId==-1)]

,PostId,CreationDate,PostType,OwnerUserId,Score,Views,Tags


## Set df to drop cascade from

In [8]:
rep_posts.shape

(21984, 8)

In [9]:
container = rep_posts[rep_posts.TagSplit.isin(top_topics.index.values)]

In [10]:
container.shape

(11172, 8)

## Set timestamps to floats

In [11]:
container.loc[:,'TimeStamp'] = container.CreationDate.apply(lambda e: time.mktime(pd.Timestamp(e).timetuple())/(10**8))

c:\Users\Joel\Anaconda2\lib\site-packages\pandas\core\indexing.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
c:\Users\Joel\Anaconda2\lib\site-packages\pandas\core\indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [12]:
## conversion preserves order (almost)
sum(~(container.sort_values(by='TimeStamp').index == container.sort_values(by='CreationDate').index))

0

## Drop cascades

In [13]:
#mapping of vertex_id () and the actual vertex (PostId)
container.reset_index(level=[0],drop=False)[[,'PostId']].to_csv('cascades-cs.txt',index=False,header=False)
with open("cascades-cs.txt","a") as fh:
    fh.write("\n")
    # Groupby attribute defines the cascades
    for k,g in container.groupby(['TagSplit']):
        df = g.sort_values(by='TimeStamp')
        fh.write(','.join(map(lambda tup:"%d,%.2f"%(tup[0],tup[1])
                     ,zip(,df.TimeStamp)))+"\n")

## Get cascade trees

In [14]:
import snap